In [19]:
import pandas as pd

# Load the 'Price' columns from both Excel files
file_path1 = 'Australia_market_1.xlsx'
file_path2 = 'stock_prices_2024_09_30.xlsx'
df1 = pd.read_excel(file_path1)[['Price']].rename(columns={'Price': 'Price_1'})
df2 = pd.read_excel(file_path2)[['Price']].rename(columns={'Price': 'Price_2'})

# Combine the two DataFrames into a single DataFrame with both 'Price_1' and 'Price_2' columns
df_combined = pd.concat([df1, df2], axis=1)

# Drop rows where 'Price_2' has null values
df_cleaned = df_combined.dropna(subset=['Price_2'])

# Save the cleaned data to a new Excel file with the sheet name "Prices"
output_path = 'combined_cleaned_prices.xlsx'
with pd.ExcelWriter(output_path) as writer:
    df_cleaned.to_excel(writer, sheet_name='Prices', index=False)

print(f"The combined and cleaned data has been saved in '{output_path}' under the sheet 'Prices'.")


The combined and cleaned data has been saved in 'combined_cleaned_prices.xlsx' under the sheet 'Prices'.


In [20]:
import pandas as pd

# Load the combined_cleaned_prices.xlsx file
df_combined = pd.read_excel('combined_cleaned_prices.xlsx', sheet_name='Prices')

# Save it as a CSV file
df_combined.to_csv('portfolio_prices.csv', index=False)

print("The file has been saved as 'portfolio_prices.csv'.")

The file has been saved as 'portfolio_prices.csv'.


In [21]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_1samp
import yfinance as yf

# Step 1: Load Portfolio Data
portfolio_prices = pd.read_csv('portfolio_prices.csv')  # Assuming first column is June 6 prices and second column is Sept 30 prices
portfolio_prices.columns = ['Price_1', 'Price_2']

# Step 2: Calculate Simple and Log Returns for Each Stock in the Portfolio
portfolio_prices['Simple_Return'] = (portfolio_prices['Price_2'] - portfolio_prices['Price_1']) / portfolio_prices['Price_1']
portfolio_prices['Log_Return'] = np.log(portfolio_prices['Price_2'] / portfolio_prices['Price_1'])

# Since the portfolio is equally weighted, calculate the average returns
average_simple_return = portfolio_prices['Simple_Return'].mean()
average_log_return = portfolio_prices['Log_Return'].mean()

# Step 3: Download Market Data for ^AORD and calculate returns over the period
stock_code = "^AORD"
start_date = "2024-06-06"
end_date = "2024-09-30"

# Download the market data
market_data = yf.download(stock_code, start=start_date, end=end_date)

# Calculate the market's simple and log return from June 6 to September 30
market_start_price = market_data['Close'].iloc[0]  # First available closing price on or after June 6
market_end_price = market_data['Close'].iloc[-1]  # Last available closing price on or before September 30
market_simple_return = (market_end_price - market_start_price) / market_start_price
market_log_return = np.log(market_end_price / market_start_price)

# Step 4: Perform One-Sample T-Tests for Simple and Log Returns against the market return
alpha = 0.05  # 95% confidence level

# T-test for Simple Return
simple_t_stat, simple_p_value = ttest_1samp(portfolio_prices['Simple_Return'], market_simple_return)

# T-test for Log Return
log_t_stat, log_p_value = ttest_1samp(portfolio_prices['Log_Return'], market_log_return)

# Print Results
print("One-Sample T-Test Results at 95% Confidence Level:")
print(f"Portfolio Average Simple Return = {average_simple_return:.4f}")
print(f"Portfolio Average Log Return = {average_log_return:.4f}")
print(f"Market Simple Return = {market_simple_return:.4f}")
print(f"Market Log Return = {market_log_return:.4f}")

print(f"Simple Return vs Market Return: T-Statistic = {simple_t_stat:.4f}, P-Value = {simple_p_value:.4f}")
print(f"Log Return vs Market Return: T-Statistic = {log_t_stat:.4f}, P-Value = {log_p_value:.4f}")

if simple_p_value < alpha:
    print("Reject the null hypothesis for Simple Return: Portfolio outperforms the market.")
else:
    print("Fail to reject the null hypothesis for Simple Return: Portfolio does not outperform the market.")

if log_p_value < alpha:
    print("Reject the null hypothesis for Log Return: Portfolio outperforms the market.")
else:
    print("Fail to reject the null hypothesis for Log Return: Portfolio does not outperform the market.")


[*********************100%***********************]  1 of 1 completed

One-Sample T-Test Results at 95% Confidence Level:
Portfolio Average Simple Return = 0.0964
Portfolio Average Log Return = 0.0095
Market Simple Return = 0.0498
Market Log Return = 0.0486
Simple Return vs Market Return: T-Statistic = 1.3964, P-Value = 0.1647
Log Return vs Market Return: T-Statistic = -0.9422, P-Value = 0.3476
Fail to reject the null hypothesis for Simple Return: Portfolio does not outperform the market.
Fail to reject the null hypothesis for Log Return: Portfolio does not outperform the market.


In [24]:
import yfinance as yf
import pandas as pd

# Load the stock list from the Excel file
stock_file_path = 'stock_prices_2024_09_30.xlsx'
stock_data = pd.read_excel(stock_file_path)
your_stock_list = stock_data['Code'].dropna().unique().tolist()  # Extract unique stock codes

# Define the start and end dates
start_date = "2024-06-06"
end_date = "2024-09-30"

# Create an empty DataFrame to store the closing prices
portfolio_daily_prices = pd.DataFrame()

# Download and collect daily closing prices for each stock
for ticker in your_stock_list:
    # Download the stock data
    data = yf.download(ticker, start=start_date, end=end_date)
    
    # Add the 'Close' column to the portfolio DataFrame, using the ticker as the column name
    portfolio_daily_prices[ticker] = data['Close']

# Save the daily prices to a CSV file
portfolio_daily_prices.to_csv('portfolio_daily_prices.csv')

print("The file 'portfolio_daily_prices.csv' has been created with daily closing prices.")


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

The file 'portfolio_daily_prices.csv' has been created with daily closing prices.



/var/folders/kq/1c1dzqtj4b7_9p_ygqzc4nfw0000gn/T/ipykernel_91342/3685174830.py:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  portfolio_daily_prices[ticker] = data['Close']


In [28]:
import pandas as pd

# Load the portfolio daily prices data
file_path = 'portfolio_daily_prices.csv'
portfolio_prices = pd.read_csv(file_path, index_col='Date', parse_dates=True)

# Drop columns (stocks) with any null values
portfolio_prices_cleaned = portfolio_prices.dropna(axis=1)

# Save the cleaned data back to CSV
portfolio_prices_cleaned.to_csv('portfolio_daily_prices.csv')

print("The cleaned file 'portfolio_daily_prices_cleaned.csv' has been saved, with all stocks containing null values removed.")


The cleaned file 'portfolio_daily_prices_cleaned.csv' has been saved, with all stocks containing null values removed.


In [29]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_1samp
import yfinance as yf

# Step 1: Load Daily Portfolio Data
# Assuming 'portfolio_daily_prices.csv' has dates as index and each stock's prices as columns
portfolio_daily_prices = pd.read_csv('portfolio_daily_prices.csv', index_col='Date', parse_dates=True)

# Step 2: Calculate Daily Simple and Log Returns for Each Stock
# Simple return: (P_t - P_(t-1)) / P_(t-1)
daily_simple_returns = portfolio_daily_prices.pct_change().dropna()

# Log return: log(P_t / P_(t-1))
daily_log_returns = np.log(portfolio_daily_prices / portfolio_daily_prices.shift(1)).dropna()

# Calculate the daily portfolio return by averaging the daily returns across all stocks
# (equally weighted)
daily_portfolio_simple_return = daily_simple_returns.mean(axis=1)
daily_portfolio_log_return = daily_log_returns.mean(axis=1)

# Step 3: Download Market Daily Data for ^AORD over the same period
stock_code = "^AORD"
start_date = daily_portfolio_simple_return.index.min().strftime('%Y-%m-%d')
end_date = daily_portfolio_simple_return.index.max().strftime('%Y-%m-%d')

# Download the market data
market_data = yf.download(stock_code, start=start_date, end=end_date)

# Calculate the market's daily simple and log returns
market_daily_simple_return = market_data['Close'].pct_change().dropna()
market_daily_log_return = np.log(market_data['Close'] / market_data['Close'].shift(1)).dropna()

# Step 4: Align Portfolio and Market Returns by Date
aligned_simple_portfolio_return, aligned_simple_market_return = daily_portfolio_simple_return.align(market_daily_simple_return, join='inner')
aligned_log_portfolio_return, aligned_log_market_return = daily_portfolio_log_return.align(market_daily_log_return, join='inner')

# Step 5: Perform One-Sample T-Tests for Daily Simple and Log Returns against the market daily return
alpha = 0.05  # 95% confidence level

# T-test for Daily Simple Returns
simple_t_stat, simple_p_value = ttest_1samp(aligned_simple_portfolio_return, aligned_simple_market_return.mean())

# T-test for Daily Log Returns
log_t_stat, log_p_value = ttest_1samp(aligned_log_portfolio_return, aligned_log_market_return.mean())

# Print Results
print("One-Sample T-Test Results at 95% Confidence Level:")
print(f"Average Daily Portfolio Simple Return = {aligned_simple_portfolio_return.mean():.4f}")
print(f"Average Daily Market Simple Return = {aligned_simple_market_return.mean():.4f}")
print(f"Simple Return vs Market Return: T-Statistic = {simple_t_stat:.4f}, P-Value = {simple_p_value:.4f}")

if simple_p_value < alpha:
    print("Reject the null hypothesis for Daily Simple Return: Portfolio outperforms the market.")
else:
    print("Fail to reject the null hypothesis for Daily Simple Return: Portfolio does not outperform the market.")

print("\n")

print(f"Average Daily Portfolio Log Return = {aligned_log_portfolio_return.mean():.4f}")
print(f"Average Daily Market Log Return = {aligned_log_market_return.mean():.4f}")
print(f"Log Return vs Market Return: T-Statistic = {log_t_stat:.4f}, P-Value = {log_p_value:.4f}")

if log_p_value < alpha:
    print("Reject the null hypothesis for Daily Log Return: Portfolio outperforms the market.")
else:
    print("Fail to reject the null hypothesis for Daily Log Return: Portfolio does not outperform the market.")


[*********************100%***********************]  1 of 1 completed

One-Sample T-Test Results at 95% Confidence Level:
Average Daily Portfolio Simple Return = 0.0016
Average Daily Market Simple Return = 0.0006
Simple Return vs Market Return: T-Statistic = 0.8731, P-Value = 0.3853
Fail to reject the null hypothesis for Daily Simple Return: Portfolio does not outperform the market.


Average Daily Portfolio Log Return = 0.0003
Average Daily Market Log Return = 0.0005
Log Return vs Market Return: T-Statistic = -0.2310, P-Value = 0.8180
Fail to reject the null hypothesis for Daily Log Return: Portfolio does not outperform the market.


In [30]:
import pandas as pd
import numpy as np
import yfinance as yf
from scipy.stats import ttest_1samp

# Assuming 'portfolio_daily_prices_cleaned.csv' is the cleaned file without null columns
portfolio_prices = pd.read_csv('portfolio_daily_prices_cleaned.csv', index_col='Date', parse_dates=True)

# Check if the DataFrame is empty
if portfolio_prices.empty:
    raise ValueError("The portfolio data is empty after cleaning.")

# Step 1: Calculate daily simple returns
daily_simple_returns = portfolio_prices.pct_change().dropna()

# Step 2: Check if the date index is valid and extract date range
if daily_simple_returns.index.isnull().any():
    raise ValueError("Invalid date index in portfolio data.")

start_date = daily_simple_returns.index.min().strftime('%Y-%m-%d')
end_date = daily_simple_returns.index.max().strftime('%Y-%m-%d')

print(f"Start date: {start_date}, End date: {end_date}")

# Step 3: Download Market Daily Data for ^AORD
stock_code = "^AORD"
market_data = yf.download(stock_code, start=start_date, end=end_date)

# Calculate market daily returns
market_daily_returns = market_data['Close'].pct_change().dropna()

# Now you can perform further analysis with daily returns
# Example: Align and compare daily returns for t-test
aligned_portfolio_return, aligned_market_return = daily_simple_returns.mean(axis=1).align(market_daily_returns, join='inner')

# T-test example
t_stat, p_value = ttest_1samp(aligned_portfolio_return, aligned_market_return.mean())

print(f"T-statistic: {t_stat}, P-value: {p_value}")


[*********************100%***********************]  1 of 1 completed

Start date: 2024-06-07, End date: 2024-09-27
T-statistic: 0.873129814022225, P-value: 0.3853071263429737


In [31]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_1samp
import yfinance as yf

# Step 1: Load and Clean Portfolio Daily Prices Data
portfolio_prices = pd.read_csv('portfolio_daily_prices_cleaned.csv', index_col='Date', parse_dates=True)

# Step 2: Calculate Daily Simple Returns for Each Stock
daily_simple_returns = portfolio_prices.pct_change().dropna()

# Step 3: Determine Start and End Dates from the Data
start_date = daily_simple_returns.index.min().strftime('%Y-%m-%d')
end_date = daily_simple_returns.index.max().strftime('%Y-%m-%d')

# Step 4: Download Market Daily Data for ^AORD
stock_code = "^AORD"
market_data = yf.download(stock_code, start=start_date, end=end_date)

# Step 5: Calculate Market Daily Returns
market_daily_returns = market_data['Close'].pct_change().dropna()

# Step 6: Calculate Portfolio's Average Daily Return (equally weighted)
daily_portfolio_return = daily_simple_returns.mean(axis=1)

# Align Portfolio and Market Returns by Date
aligned_portfolio_return, aligned_market_return = daily_portfolio_return.align(market_daily_returns, join='inner')

# Step 7: Perform One-Sample T-Test
alpha = 0.05  # 95% confidence level

# Calculate t-statistic and p-value
t_stat, p_value = ttest_1samp(aligned_portfolio_return, aligned_market_return.mean())

# Step 8: Print Results
print("One-Sample T-Test Results at 95% Confidence Level:")
print(f"Average Daily Portfolio Return = {aligned_portfolio_return.mean():.4f}")
print(f"Average Daily Market Return = {aligned_market_return.mean():.4f}")
print(f"T-Statistic = {t_stat:.4f}, P-Value = {p_value:.4f}")

# Determine if portfolio outperforms the market
if p_value < alpha:
    print("Reject the null hypothesis: Portfolio daily returns are significantly different from the market's daily returns.")
    if aligned_portfolio_return.mean() > aligned_market_return.mean():
        print("The portfolio outperforms the market.")
    else:
        print("The portfolio underperforms compared to the market.")
else:
    print("Fail to reject the null hypothesis: Portfolio daily returns are not significantly different from the market's daily returns.")


[*********************100%***********************]  1 of 1 completed

One-Sample T-Test Results at 95% Confidence Level:
Average Daily Portfolio Return = 0.0016
Average Daily Market Return = 0.0006
T-Statistic = 0.8731, P-Value = 0.3853
Fail to reject the null hypothesis: Portfolio daily returns are not significantly different from the market's daily returns.


In [35]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_1samp
import yfinance as yf

# Step 1: Load and Clean Portfolio Daily Prices Data
portfolio_prices = pd.read_csv('portfolio_daily_prices_cleaned.csv', index_col='Date', parse_dates=True)

# Step 2: Calculate Daily Simple and Log Returns for Each Stock
# Simple Return: (P_t - P_(t-1)) / P_(t-1)
daily_simple_returns = portfolio_prices.pct_change().dropna()

# Log Return: log(P_t / P_(t-1))
daily_log_returns = np.log(portfolio_prices / portfolio_prices.shift(1)).dropna()

# Step 3: Determine Start and End Dates from the Data
start_date = daily_simple_returns.index.min().strftime('%Y-%m-%d')
end_date = daily_simple_returns.index.max().strftime('%Y-%m-%d')

# Step 4: Download Market Daily Data for ^AORD
stock_code = "^AORD"
market_data = yf.download(stock_code, start=start_date, end=end_date)

# Step 5: Calculate Market Daily Simple and Log Returns
market_daily_simple_return = market_data['Close'].pct_change().dropna()
market_daily_log_return = np.log(market_data['Close'] / market_data['Close'].shift(1)).dropna()

# Step 6: Calculate Portfolio's Average Daily Returns (equally weighted)
# Daily portfolio return is the mean across all stocks for each day
daily_portfolio_simple_return = daily_simple_returns.mean(axis=1)
daily_portfolio_log_return = daily_log_returns.mean(axis=1)

# Align Portfolio and Market Returns by Date
aligned_simple_portfolio_return, aligned_simple_market_return = daily_portfolio_simple_return.align(market_daily_simple_return, join='inner')
aligned_log_portfolio_return, aligned_log_market_return = daily_portfolio_log_return.align(market_daily_log_return, join='inner')

# Step 7: Perform One-Sample T-Tests for Simple and Log Returns
alpha = 0.05  # 95% confidence level

# T-test for Daily Simple Returns
simple_t_stat, simple_p_value = ttest_1samp(aligned_simple_portfolio_return, aligned_simple_market_return.mean())

# T-test for Daily Log Returns
log_t_stat, log_p_value = ttest_1samp(aligned_log_portfolio_return, aligned_log_market_return.mean())

# Step 8: Print Results
print("One-Sample T-Test Results at 95% Confidence Level:")
print(f"\nSimple Returns:")
print(f"Average Daily Portfolio Simple Return = {aligned_simple_portfolio_return.mean():.4f}")
print(f"Average Daily Market Simple Return = {aligned_simple_market_return.mean():.4f}")
print(f"Simple Return T-Statistic = {simple_t_stat:.4f}, P-Value = {simple_p_value:.4f}")

if simple_p_value < alpha:
    print("Reject the null hypothesis for Simple Returns: Portfolio daily simple returns are significantly different from the market's.")
    if aligned_simple_portfolio_return.mean() > aligned_simple_market_return.mean():
        print("The portfolio outperforms the market in terms of daily simple returns.")
    else:
        print("The portfolio underperforms compared to the market in terms of daily simple returns.")
else:
    print("Fail to reject the null hypothesis for Simple Returns: Portfolio daily simple returns are not significantly different from the market's.")

print(f"\nLog Returns:")
print(f"Average Daily Portfolio Log Return = {aligned_log_portfolio_return.mean():.4f}")
print(f"Average Daily Market Log Return = {aligned_log_market_return.mean():.4f}")
print(f"Log Return T-Statistic = {log_t_stat:.4f}, P-Value = {log_p_value:.4f}")

if log_p_value < alpha:
    print("Reject the null hypothesis for Log Returns: Portfolio daily log returns are significantly different from the market's.")
    if aligned_log_portfolio_return.mean() > aligned_log_market_return.mean():
        print("The portfolio outperforms the market in terms of daily log returns.")
    else:
        print("The portfolio underperforms compared to the market in terms of daily log returns.")
else:
    print("Fail to reject the null hypothesis for Log Returns: Portfolio daily log returns are not significantly different from the market's.")


[*********************100%***********************]  1 of 1 completed

One-Sample T-Test Results at 95% Confidence Level:

Simple Returns:
Average Daily Portfolio Simple Return = 0.0016
Average Daily Market Simple Return = 0.0006
Simple Return T-Statistic = 0.8731, P-Value = 0.3853
Fail to reject the null hypothesis for Simple Returns: Portfolio daily simple returns are not significantly different from the market's.

Log Returns:
Average Daily Portfolio Log Return = 0.0003
Average Daily Market Log Return = 0.0005
Log Return T-Statistic = -0.2310, P-Value = 0.8180
Fail to reject the null hypothesis for Log Returns: Portfolio daily log returns are not significantly different from the market's.


In [36]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_1samp
import yfinance as yf

# Step 1: Load Portfolio Prices on June 6 and September 30
# Assuming 'portfolio_prices.csv' has June 6 prices in 'Price_1' column and Sept 30 prices in 'Price_2' column
portfolio_prices = pd.read_csv('portfolio_prices.csv')
portfolio_prices.columns = ['Price_1', 'Price_2']

# Step 2: Calculate Cumulative Simple and Log Returns for Each Stock
portfolio_prices['Cumulative_Simple_Return'] = (portfolio_prices['Price_2'] - portfolio_prices['Price_1']) / portfolio_prices['Price_1']
portfolio_prices['Cumulative_Log_Return'] = np.log(portfolio_prices['Price_2'] / portfolio_prices['Price_1'])

# Calculate the average cumulative simple and log returns across all stocks (equally weighted)
average_cumulative_simple_return = portfolio_prices['Cumulative_Simple_Return'].mean()
average_cumulative_log_return = portfolio_prices['Cumulative_Log_Return'].mean()

# Step 3: Download Market Data for ^AORD and calculate cumulative return from June 6 to September 30
stock_code = "^AORD"
start_date = "2024-06-06"
end_date = "2024-09-30"

# Download the market data
market_data = yf.download(stock_code, start=start_date, end=end_date)

# Calculate the market's cumulative simple and log return
market_start_price = market_data['Close'].iloc[0]
market_end_price = market_data['Close'].iloc[-1]
market_cumulative_simple_return = (market_end_price - market_start_price) / market_start_price
market_cumulative_log_return = np.log(market_end_price / market_start_price)

# Step 4: Perform One-Sample T-Tests for Cumulative Simple and Log Returns
alpha = 0.05  # 95% confidence level

# T-test for Cumulative Simple Return
simple_t_stat, simple_p_value = ttest_1samp(portfolio_prices['Cumulative_Simple_Return'], market_cumulative_simple_return)

# T-test for Cumulative Log Return
log_t_stat, log_p_value = ttest_1samp(portfolio_prices['Cumulative_Log_Return'], market_cumulative_log_return)

# Step 5: Print Results
print("One-Sample T-Test Results at 95% Confidence Level (Cumulative Returns):")
print(f"\nCumulative Simple Returns:")
print(f"Average Cumulative Portfolio Simple Return = {average_cumulative_simple_return:.4f}")
print(f"Market Cumulative Simple Return = {market_cumulative_simple_return:.4f}")
print(f"Simple Return T-Statistic = {simple_t_stat:.4f}, P-Value = {simple_p_value:.4f}")

if simple_p_value < alpha:
    print("Reject the null hypothesis for Cumulative Simple Returns: Portfolio cumulative simple returns are significantly different from the market's.")
    if average_cumulative_simple_return > market_cumulative_simple_return:
        print("The portfolio outperforms the market in terms of cumulative simple returns.")
    else:
        print("The portfolio underperforms compared to the market in terms of cumulative simple returns.")
else:
    print("Fail to reject the null hypothesis for Cumulative Simple Returns: Portfolio cumulative simple returns are not significantly different from the market's.")

print(f"\nCumulative Log Returns:")
print(f"Average Cumulative Portfolio Log Return = {average_cumulative_log_return:.4f}")
print(f"Market Cumulative Log Return = {market_cumulative_log_return:.4f}")
print(f"Log Return T-Statistic = {log_t_stat:.4f}, P-Value = {log_p_value:.4f}")

if log_p_value < alpha:
    print("Reject the null hypothesis for Cumulative Log Returns: Portfolio cumulative log returns are significantly different from the market's.")
    if average_cumulative_log_return > market_cumulative_log_return:
        print("The portfolio outperforms the market in terms of cumulative log returns.")
    else:
        print("The portfolio underperforms compared to the market in terms of cumulative log returns.")
else:
    print("Fail to reject the null hypothesis for Cumulative Log Returns: Portfolio cumulative log returns are not significantly different from the market's.")


[*********************100%***********************]  1 of 1 completed

One-Sample T-Test Results at 95% Confidence Level (Cumulative Returns):

Cumulative Simple Returns:
Average Cumulative Portfolio Simple Return = 0.0964
Market Cumulative Simple Return = 0.0498
Simple Return T-Statistic = 1.3964, P-Value = 0.1647
Fail to reject the null hypothesis for Cumulative Simple Returns: Portfolio cumulative simple returns are not significantly different from the market's.

Cumulative Log Returns:
Average Cumulative Portfolio Log Return = 0.0095
Market Cumulative Log Return = 0.0486
Log Return T-Statistic = -0.9422, P-Value = 0.3476
Fail to reject the null hypothesis for Cumulative Log Returns: Portfolio cumulative log returns are not significantly different from the market's.


In [37]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_1samp
import yfinance as yf

# Load Portfolio Prices on June 6 and September 30
portfolio_prices = pd.read_csv('portfolio_prices.csv')
portfolio_prices.columns = ['Price_1', 'Price_2']

# Calculate Cumulative Simple and Log Returns for Each Stock
portfolio_prices['Cumulative_Simple_Return'] = (portfolio_prices['Price_2'] - portfolio_prices['Price_1']) / portfolio_prices['Price_1']
portfolio_prices['Cumulative_Log_Return'] = np.log(portfolio_prices['Price_2'] / portfolio_prices['Price_1'])

# Number of days from June 6 to September 30
num_days = 82

# Calculate the average daily simple and log returns for the portfolio
portfolio_prices['Daily_Simple_Return'] = (1 + portfolio_prices['Cumulative_Simple_Return'])**(1 / num_days) - 1
portfolio_prices['Daily_Log_Return'] = portfolio_prices['Cumulative_Log_Return'] / num_days

# Calculate the average daily simple and log returns across all stocks (equally weighted)
average_daily_simple_return = portfolio_prices['Daily_Simple_Return'].mean()
average_daily_log_return = portfolio_prices['Daily_Log_Return'].mean()

# Download Market Data for ^AORD and calculate cumulative return
stock_code = "^AORD"
start_date = "2024-06-06"
end_date = "2024-09-30"
market_data = yf.download(stock_code, start=start_date, end=end_date)

# Market cumulative simple and log returns
market_start_price = market_data['Close'].iloc[0]
market_end_price = market_data['Close'].iloc[-1]
market_cumulative_simple_return = (market_end_price - market_start_price) / market_start_price
market_cumulative_log_return = np.log(market_end_price / market_start_price)

# Calculate the market's daily simple and log returns
market_daily_simple_return = (1 + market_cumulative_simple_return)**(1 / num_days) - 1
market_daily_log_return = market_cumulative_log_return / num_days

# Perform T-tests
alpha = 0.05  # 95% confidence level

# T-test for Daily Simple Returns
simple_t_stat, simple_p_value = ttest_1samp(portfolio_prices['Daily_Simple_Return'], market_daily_simple_return)

# T-test for Daily Log Returns
log_t_stat, log_p_value = ttest_1samp(portfolio_prices['Daily_Log_Return'], market_daily_log_return)

# Print Results
print("One-Sample T-Test Results at 95% Confidence Level (Daily Returns):")
print(f"\nDaily Simple Returns:")
print(f"Average Daily Portfolio Simple Return = {average_daily_simple_return:.4f}")
print(f"Market Daily Simple Return = {market_daily_simple_return:.4f}")
print(f"Simple Return T-Statistic = {simple_t_stat:.4f}, P-Value = {simple_p_value:.4f}")

if simple_p_value < alpha:
    print("Reject the null hypothesis for Daily Simple Returns: Portfolio daily simple returns are significantly different from the market's.")
    if average_daily_simple_return > market_daily_simple_return:
        print("The portfolio outperforms the market in terms of daily simple returns.")
    else:
        print("The portfolio underperforms compared to the market in terms of daily simple returns.")
else:
    print("Fail to reject the null hypothesis for Daily Simple Returns: Portfolio daily simple returns are not significantly different from the market's.")

print(f"\nDaily Log Returns:")
print(f"Average Daily Portfolio Log Return = {average_daily_log_return:.4f}")
print(f"Market Daily Log Return = {market_daily_log_return:.4f}")
print(f"Log Return T-Statistic = {log_t_stat:.4f}, P-Value = {log_p_value:.4f}")

if log_p_value < alpha:
    print("Reject the null hypothesis for Daily Log Returns: Portfolio daily log returns are significantly different from the market's.")
    if average_daily_log_return > market_daily_log_return:
        print("The portfolio outperforms the market in terms of daily log returns.")
    else:
        print("The portfolio underperforms compared to the market in terms of daily log returns.")
else:
    print("Fail to reject the null hypothesis for Daily Log Returns: Portfolio daily log returns are not significantly different from the market's.")


[*********************100%***********************]  1 of 1 completed

One-Sample T-Test Results at 95% Confidence Level (Daily Returns):

Daily Simple Returns:
Average Daily Portfolio Simple Return = 0.0001
Market Daily Simple Return = 0.0006
Simple Return T-Statistic = -0.9170, P-Value = 0.3606
Fail to reject the null hypothesis for Daily Simple Returns: Portfolio daily simple returns are not significantly different from the market's.

Daily Log Returns:
Average Daily Portfolio Log Return = 0.0001
Market Daily Log Return = 0.0006
Log Return T-Statistic = -0.9422, P-Value = 0.3476
Fail to reject the null hypothesis for Daily Log Returns: Portfolio daily log returns are not significantly different from the market's.
